In [1]:
import requests
from bs4 import BeautifulSoup

In [31]:
def is_a_valid_recipe(soup):
    title = soup.title.string != ''
    ingredients = any(map(
        lambda x: 'class' in x.attrs and 'section-title' in x['class'] and x.string.strip() == 'Ingredients',
        soup.find_all('h2')
    ))
    instructions = any(map(
        lambda x: 'class' in x.attrs and 'section-title' in x['class'] and x.string.strip() == 'Preparation',
        soup.find_all('h2')
    ))
    return title and ingredients and instructions

In [ ]:
%%time

total_urls  = 0
total_recipes = 0
recipe_errors = 0
response_errors = 0

retry_list = []

base_url = 'http://www.epicurious.com/recipes/food/views/'
max_id = 6009
for ide in range(1, max_id + 1):
    try:
        str_id = str(ide)
        url = base_url + str_id
        response = requests.get(url)
        if response.ok:
            soup = BeautifulSoup(response.content, 'html.parser')
            if is_a_valid_recipe(soup):
                with open('data/epicurious/' + str_id + '.html', 'w') as f:
                    f.write(soup.prettify())
                    total_recipes += 1
            else:
                recipe_errors += 1
        else:
            response_errors += 1
        except:
            retry_list.append(ide)
    total_urls += 1
    if total_urls % 1000 == 0:
        print(total_urls, 'processed.')

print('Total urls:', total_urls)
print('Total recipes:', total_recipes)
print('Recipe errors:', recipe_errors)
print('Response errors:', response_errors)